 ## Predicting movie gross revenue 

#### Read movie data

In [161]:
import pandas as pd
files = !ls *.csv 

    
final_data = pd.concat([pd.read_csv(f, index_col=0, header=None, skiprows = 1) for f in files])


In [165]:
final_data = final_data.reset_index(drop=True)

In [166]:
final_data.columns = ["Release_Date","Movie","Gross","Genre","Run_Time","Rating","Budget","Num_theatres","Actors","Producer","Writer","Director"]

In [167]:
final_data.head()

,Release_Date,Movie,Gross,Genre,Run_Time,Rating,Budget,Num_theatres,Actors,Producer,Writer,Director
0,"April 9, 2004",The Alamo (2004),"$22,414,961",War,2 hrs. 17 min.,PG-13,$107 million,"2,609 theaters","['Billy Bob Thornton', 'Dennis Quaid', 'Patric...",Ron Howard,Leslie Bohem,John Lee Hancock
1,"December 25, 2004",Darkness,"$22,163,442",Horror,1 hrs. 42 min.,PG-13,$10.6 million,"1,718 theaters","['Anna Paquin', 'Lena Olin']",NaN,NaN,NaN
2,"September 17, 2004",Mr. 3000,"$21,811,187",Sports Comedy,1 hrs. 44 min.,PG-13,NaN,"2,736 theaters","['Keegan-Michael Key', 'Bernie Mac', 'Angela B...",Gary Barber,NaN,Charles Stone III
3,"March 12, 2004",NASCAR 3D:\nThe IMAX Experience (IMAX),"$21,583,831",IMAX,0 hrs. 40 min.,PG,NaN,73 theaters,[],NaN,NaN,NaN
4,"January 16, 2004",Torque,"$21,215,059",Action,1 hrs. 21 min.,PG-13,$40 million,"2,463 theaters","['Ice Cube', 'Adam Scott']",Neal H. Moritz,NaN,NaN


### Cleaning

In [169]:
final_data = final_data.dropna(how='all')

In [171]:
final_data = final_data.dropna(subset=["Budget"])

In [176]:
final_data = final_data[final_data["Genre"] != "Documentary"]

In [177]:
final_data = final_data[final_data["Genre"] !="Animation"]

In [179]:
final_data["Movie"] = final_data["Movie"].str.replace("\n","")

In [180]:
def to_minutes(hours):
    if isinstance(hours,float):
        return hours
    hr = [int(s) for s in hours.split() if s.isdigit()]
   
    minutes = hr[0]*60 + hr[1]
    return minutes
final_data["Run_Time"]=final_data["Run_Time"].apply(to_minutes)

In [181]:
from datetime import datetime
from datetime import timedelta
final_data["Release_Date"] = final_data["Release_Date"].apply(lambda x: datetime.strptime(x,"%B %d, %Y") )

In [184]:
final_data["Gross"] = final_data["Gross"].str.strip('$').str.replace(",","").str.replace("Estimate","").str.replace("(","").str.replace(")","").str.strip(" ")
final_data["Gross"] = final_data["Gross"].astype(int)

In [185]:

final_data["Num_theatres"] = final_data["Num_theatres"].str.replace(",","").str.replace("theaters","")
final_data["Num_theatres"] = final_data["Num_theatres"].astype(int)

In [186]:
import math
def transform_budget(budget):
    if isinstance(budget,float):
        
        return budget
    if 'million' in budget:
        return float(budget.strip("$").replace("million",""))*1000000
    else:
        return float(budget.strip("$").replace(",",""))
final_data["Budget"] = final_data["Budget"].apply(transform_budget)

In [188]:
final_data.loc[(final_data["Genre"] == "Romantic Comedy") | (final_data["Genre"] == "Family Comedy") | (final_data["Genre"] == "Action Comedy")|
          (final_data["Genre"] == "Horror Comedy")|(final_data["Genre"] == "Sports Comedy")|(final_data["Genre"] == "Crime Comedy")|(final_data["Genre"] == "Adventure Comedy")
               |(final_data["Genre"] == "Western Comedy"),"Genre"] = "Comedy" 

In [189]:
final_data.loc[(final_data["Genre"] == "Comedy / Drama") | (final_data["Genre"] == "Period Drama") | (final_data["Genre"] == "Drama / Thriller")|
          (final_data["Genre"] == "Crime Drama")|(final_data["Genre"] == "Sports Drama")|(final_data["Genre"] == " Music Drama")|(final_data["Genre"] == "Action Drama")|
             (final_data["Genre"] == "Family")|(final_data["Genre"] == "Musical")|
              (final_data["Genre"] == "Historical Drama")|(final_data["Genre"] == "Western")|
               (final_data["Genre"] == "War")|(final_data["Genre"] == "War Drama")|
               (final_data["Genre"] == "Historical Epic")|(final_data["Genre"] == "Music Drama")
               |(final_data["Genre"] == "Concert"),"Genre"] = "Drama" 

In [190]:
final_data.loc[(final_data["Genre"] == "Action Horror") | (final_data["Genre"] == "Horror Thriller") | (final_data["Genre"] == "Period Horror"),"Genre"] = "Horror" 

In [191]:
final_data.loc[(final_data["Genre"] == "Crime Thriller") | (final_data["Genre"] == "Comedy Thriller") | (final_data["Genre"] == "Romantic Thriller"),"Genre"] = "Thriller" 

In [192]:
final_data.loc[(final_data["Genre"] == "Action / Adventure") | (final_data["Genre"] == "Action Thriller") | (final_data["Genre"] == "Action / Crime")|
          (final_data["Genre"] == "Period Action")|(final_data["Genre"] == "Crime"),"Genre"] = "Action" 

In [193]:
final_data.loc[(final_data["Genre"] == "Family Adventure") | (final_data["Genre"] == "Period Adventure") | (final_data["Genre"] == "Romantic Adventure"),"Genre"] = "Adventure" 

In [194]:
final_data.loc[(final_data["Genre"] == "Fantasy Drama") | (final_data["Genre"] == "Fantasy Comedy") | (final_data["Genre"] == "Action Fantasy"),"Genre"] = "Fantasy" 

In [195]:
final_data.loc[(final_data["Genre"] == "Sci-Fi Comedy") | (final_data["Genre"] == "Sci-Fi Fantasy") | (final_data["Genre"] == "Sci-Fi Horror")|
          (final_data["Genre"] == "Sci-Fi Adventure")|(final_data["Genre"] == "Sci-Fi Action")|(final_data["Genre"] == "Sci-Fi Thriller"),"Genre"] = "Sci-Fi" 

In [196]:
final_data.loc[(final_data["Genre"] == "Foreign / Action") | (final_data["Genre"] == "Foreign / Horror"),"Genre"] = "Foreign" 

In [197]:
final_data = final_data.reset_index(drop=True)

In [201]:
for i in range(len(final_data)):
    item= final_data.ix[i,"Release_Date"]
    range_max = item + timedelta(days=7)
    range_min = item - timedelta(days=7)
    result = final_data[final_data["Release_Date"]<=range_max]
    final_res = len(result[range_min<=final_data["Release_Date"]])
    final_data.ix[i,"Competitors"] = final_res

/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [202]:
import string
def strip_punctuation(line):
    line1 = "".join(l for l in line if l not in string.punctuation)
    return line1.replace(" ","")
final_data["clean_movie"] = final_data["Movie"].apply(strip_punctuation)

In [ ]:
def get_month(x):
    return x.strftime('%B')
final_data["month"] = final_data["Release_Date"].apply(get_month)

In [ ]:
def get_year(x):
    return x.strftime('%Y')
final_data["year"] = final_data["Release_Date"].apply(get_year)

### Adjust Gross Revenue for inflation

In [ ]:
def inflation_adjust(x):
    inflation = {
    2004: 1.27,
    2005: 1.23,
    2006: 1.19,
    2007: 1.16,
    2008: 1.12,
    2009: 1.12,
    2010: 1.10,
    2011: 1.07,
    2012: 1.05,
    2013: 1.03,
    2014: 1.01,
    2015: 1.01,
    2016: 1.00
    }
    return float(inflation[int(x)])
final_data["Gross_inflate"] = final_data["Gross"].mul(final_data["year"].apply(inflation_adjust))
final_data["Budget_inflate"] = final_data["Budget"].mul(final_data["year"].apply(inflation_adjust))

### Read Actors' data 

In [204]:
actors = pd.read_csv("/Users/Manoj/Documents/metis/actors.csv",converters={"movies":lambda x:x.strip("[]").split(",")})

In [206]:
actor_temp = pd.DataFrame(columns=["names","movies"])
for i in range(len(actors)):
    item=actors.ix[i,"movies"]
    
    
    for j in range(len(item)):
        mov = item[j].strip(" ").strip("'").strip("\"")
        temp = pd.DataFrame({'names':actors.ix[i,"names"],'movies':mov},index=[0])
        actor_temp = actor_temp.append(temp)
actor_temp = actor_temp.reset_index(drop=True)
print(actor_temp.head(n=40))

                                            movies        names
0               Batman v Superman: Dawn of Justice    Amy Adams
1                                         Big Eyes    Amy Adams
2                                          Lullaby    Amy Adams
3                                       Her (2013)    Amy Adams
4                                  American Hustle    Amy Adams
5                                     Man of Steel    Amy Adams
6                                      On the Road    Amy Adams
7                           Trouble with the Curve    Amy Adams
8                                       The Master    Amy Adams
9                                      The Muppets    Amy Adams
10                                     The Fighter    Amy Adams
11                                       Leap Year    Amy Adams
12                                   Julie & Julia    Amy Adams
13  Night at the Museum: Battle of the Smithsonian    Amy Adams
14                               Sunshin

In [208]:

actor_temp["clean_movie"] = actor_temp["movies"].apply(strip_punctuation)



In [210]:
temp_dict = {}
for i in range(len(final_data)):
    item = final_data.ix[i,"clean_movie"]
    item = item.strip(" ")
    
    if any(actor_temp[actor_temp["clean_movie"].str.strip(" ") == (item)]):
        row = actor_temp[actor_temp["clean_movie"] == (item)].index.tolist()
        if len(row) not in temp_dict:
            temp_dict[len(row)] = 1
        else:
            temp_dict[len(row)] += 1
        
            
        
        actor_temp.ix[row,"Gross"] = final_data.ix[i,"Gross"]
        actor_temp.ix[row,"Release_Date"] = final_data.ix[i,"Release_Date"]
        
print(temp_dict)
    

{0: 230, 1: 196, 2: 234, 3: 262, 4: 239, 5: 149, 6: 112, 7: 57, 8: 30, 9: 18, 10: 9, 11: 6, 12: 4, 13: 4, 14: 1, 19: 1}


In [211]:
actor_temp.head(n=4)

,movies,names,clean_movie,Gross,Release_Date
0,Batman v Superman: Dawn of Justice,Amy Adams,BatmanvSupermanDawnofJustice,330360194.0,2016-03-25
1,Big Eyes,Amy Adams,BigEyes,NaN,NaT
2,Lullaby,Amy Adams,Lullaby,NaN,NaT
3,Her (2013),Amy Adams,Her2013,25568251.0,2013-12-18


In [212]:
actor_temp = actor_temp.dropna(subset=["Gross"])

In [213]:
len(actor_temp)

4913

### Average Gross Revenue of the actors' at each movie level

In [214]:
actor_final = pd.DataFrame(columns=["name","Movie","Release_Date","Average_Gross","clean_movie"])
for name, group in actor_temp.groupby(actor_temp["names"]):
    
    group.sort_values(by=['Release_Date'],ascending=True,inplace=True)
    group = group.reset_index(drop=True)
    group["cum_sum"] = group["Gross"].cumsum()
    group["Average_Gross"] = [group.ix[i,'cum_sum']/(i+1) for i in range(len(group))]
    for i in range(len(group)):
        temp = pd.DataFrame({'name':[name],'clean_movie':[group.ix[i,"clean_movie"]],'Movie':[group.ix[i,"movies"]],'Gross':[group.ix[i,"Gross"]],'Release_Date':[group.ix[i,"Release_Date"]],'Average_Gross':[group.ix[i,"Average_Gross"]]})
                           
        actor_final=actor_final.append(temp)

/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [215]:
actor_final.head()

,Average_Gross,Gross,Movie,Release_Date,clean_movie,name
0,8725813.0,8725813.0,Suspect Zero,2004-08-27,SuspectZero,Aaron Eckhart
0,16759661.0,24793509.0,Thank You for Smoking,2006-03-17,ThankYouforSmoking,Aaron Eckhart
0,11299580.0,379418.0,Conversations with Other Women,2006-08-11,ConversationswithOtherWomen,Aaron Eckhart
0,14110955.0,22545080.0,The Black Dahlia,2006-09-15,TheBlackDahlia,Aaron Eckhart
0,117957835.6,533345358.0,The Dark Knight,2008-07-18,TheDarkKnight,Aaron Eckhart


In [216]:
actor_final = actor_final.reset_index(drop=True)

In [222]:
actor_final.to_csv("/Users/Manoj/Documents/metis/actor_final.csv")

In [220]:
final_data["Actor1"] = 0
final_data["Actor2"] = 0
final_data["Actor3"] = 0
for i in range(len(final_data)):
    if isinstance(final_data.ix[i,"Actors"],float):
         continue
    name = final_data.ix[i,"Actors"].strip("[]").split(",")
    movie = final_data.ix[i,"clean_movie"]
   
    for j in range(len(name)):
        temp = name[j].strip(" ").strip("'").strip("\"")
        
       
        if any(actor_final[(actor_final["name"].str.contains(temp)) & (actor_final["clean_movie"] == (movie))]):
            row = actor_final[(actor_final["name"].str.contains(temp)) & (actor_final["clean_movie"]==(movie))].index.tolist()
            if len(row) == 0:
                continue
            if len(row) > 1:
                print("More than one match",row)
            else:
                if j == 0:
                    
                    final_data.ix[i,"Actor1"] = actor_final.ix[row[0],"Average_Gross"]
                elif j==1:
                    
                    final_data.ix[i,"Actor2"] = actor_final.ix[row[0],"Average_Gross"]
                else:
                    
                    final_data.ix[i,"Actor3"] = actor_final.ix[row[0],"Average_Gross"]

                    
                    
                
           


/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [221]:
final_data[:10]

,Release_Date,Movie,Gross,Genre,Run_Time,Rating,Budget,Num_theatres,Actors,Producer,Writer,Director,ROI,Competitors,clean_movie,Actor1,Actor2,Actor3
0,2004-04-09,The Alamo (2004),22414961,Drama,137,PG-13,107000000.0,2609,"['Billy Bob Thornton', 'Dennis Quaid', 'Patric...",Ron Howard,Leslie Bohem,John Lee Hancock,-0.790514,13.0,TheAlamo2004,2.241496e+07,22414961.0,22414961.0
1,2004-12-25,Darkness,22163442,Horror,102,PG-13,10600000.0,1718,"['Anna Paquin', 'Lena Olin']",NaN,NaN,NaN,1.090891,4.0,Darkness,2.216344e+07,0.0,0.0
2,2004-01-16,Torque,21215059,Action,81,PG-13,40000000.0,2463,"['Ice Cube', 'Adam Scott']",Neal H. Moritz,NaN,NaN,-0.469624,7.0,Torque,2.121506e+07,21215059.0,0.0
3,2004-12-17,Flight of the Phoenix,21009180,Adventure,120,PG-13,45000000.0,2604,"['Dennis Quaid', 'Tyrese Gibson', 'Giovanni Ri...",John Davis,Scott Frank,John Moore,-0.533129,9.0,FlightofthePhoenix,7.672165e+07,21009180.0,29385928.5
4,2004-08-06,Little Black Book,20698668,Comedy,105,PG-13,35000000.0,2445,"['Brittany Murphy', 'Holly Hunter', 'Ron Livin...",NaN,Elisa Bell,Nick Hurran,-0.408609,11.0,LittleBlackBook,2.069867e+07,20698668.0,20698668.0
5,2004-07-30,Harold and Kumar Go to White Castle,18250550,Comedy,88,R,9000000.0,2163,"['John Cho', 'Kal Penn', 'Neil Patrick Harris']",David Brewington (executive),Jon Hurwitz,NaN,1.027839,12.0,HaroldandKumarGotoWhiteCastle,0.000000e+00,18250550.0,18250550.0
6,2004-04-30,Laws of Attraction,17871255,Comedy,87,PG-13,32000000.0,2449,"['Pierce Brosnan', 'Julianne Moore', 'Michael ...",Mark Gordon (executive),Aline Brosh McKenna,NaN,-0.441523,11.0,LawsofAttraction,1.787126e+07,17871255.0,17871255.0
7,2004-02-20,Eurotrip,17771387,Comedy,92,R,25000000.0,2544,"['Michelle Trachtenberg', 'Matt Damon* (Cameo)']",Alec Berg,Alec Berg,Jeff Schaffer,-0.289145,6.0,Eurotrip,1.777139e+07,0.0,0.0
8,2004-01-09,My Baby's Daddy,17669317,Comedy,86,PG-13,12000000.0,1448,NaN,Peter Safran (executive),NaN,NaN,0.472443,5.0,MyBabysDaddy,0.000000e+00,0.0,0.0
9,2004-11-12,Seed of Chucky,17083732,Comedy,87,R,12000000.0,2062,[],NaN,NaN,NaN,0.423644,9.0,SeedofChucky,0.000000e+00,0.0,0.0


### Reading and cleaning directors' data

In [223]:
directors = pd.read_csv("/Users/Manoj/Documents/metis/directors.csv",converters={"movies":lambda x:x.strip("[]").split(",")})

In [224]:
directors.head(n=5)

,Unnamed: 0,names,movies
0,0,Marc Abraham,"['I Saw the Light', 'Flash of Genius']"
1,1,Jim Abrahams,"['Mafia!', 'Hot Shots! Part Deux', 'Hot Shot..."
2,2,J.J. Abrams,"['Star Wars: The Force Awakens', 'Star Trek I..."
3,3,Andrew Adamson,"['Mr. Pip', 'Cirque Du Soleil: Worlds Away', ..."
4,4,Gilbert Adler,['Bordello of Blood']


In [225]:
type(directors.ix[0,"movies"])

list

In [226]:
director_temp = pd.DataFrame(columns=["names","movies"])
for i in range(len(directors)):
    item=directors.ix[i,"movies"]
    
    
    for j in range(len(item)):
        mov = item[j].strip(" ").strip("'").strip("\"")
        temp = pd.DataFrame({'names':directors.ix[i,"names"],'movies':mov},index=[0])
        director_temp = director_temp.append(temp)
director_temp = director_temp.reset_index(drop=True)


In [227]:
director_temp[:4]

,movies,names
0,I Saw the Light,Marc Abraham
1,Flash of Genius,Marc Abraham
2,Mafia!,Jim Abrahams
3,Hot Shots! Part Deux,Jim Abrahams


In [228]:
director_temp["clean_movie"] = director_temp["movies"].apply(strip_punctuation)



In [229]:

for i in range(len(final_data)):
    item = final_data.ix[i,"clean_movie"]
    item = item.strip(" ")
    
    if any(director_temp[director_temp["clean_movie"].str.strip(" ") == (item)]):
        row = director_temp[director_temp["clean_movie"] == (item)].index.tolist()
       
        
            
        
        director_temp.ix[row,"Gross"] = final_data.ix[i,"Gross"]
        director_temp.ix[row,"Release_Date"] = final_data.ix[i,"Release_Date"]
        

    

In [230]:
director_temp[:15]

,movies,names,clean_movie,Gross,Release_Date
0,I Saw the Light,Marc Abraham,ISawtheLight,NaN,NaT
1,Flash of Genius,Marc Abraham,FlashofGenius,NaN,NaT
2,Mafia!,Jim Abrahams,Mafia,NaN,NaT
3,Hot Shots! Part Deux,Jim Abrahams,HotShotsPartDeux,NaN,NaT
4,Hot Shots!,Jim Abrahams,HotShots,NaN,NaT
5,Welcome Home,Jim Abrahams,WelcomeHome,NaN,NaT
6,Roxy Carmichael,Jim Abrahams,RoxyCarmichael,NaN,NaT
7,Big Business,Jim Abrahams,BigBusiness,NaN,NaT
8,Ruthless People,Jim Abrahams,RuthlessPeople,NaN,NaT
9,Top Secret!,Jim Abrahams,TopSecret,NaN,NaT


In [231]:
director_temp = director_temp.dropna(subset=["Gross"])

In [244]:
director_final = pd.DataFrame(columns=["name","Movie","Release_Date","Average_Gross","clean_movie"])
for name, group in director_temp.groupby(director_temp["names"]):
    
    group.sort_values(by=['Release_Date'],ascending=True,inplace=True)
    group = group.reset_index(drop=True)
    group["cum_sum"] = group["Gross"].cumsum()
    group["Average_Gross"] = [group.ix[i,'cum_sum']/(i+1) for i in range(len(group))]
    for i in range(len(group)):
        temp = pd.DataFrame({'name':[name],'clean_movie':[group.ix[i,"clean_movie"]],'Movie':[group.ix[i,"movies"]],'Gross':[group.ix[i,"Gross"]],'Release_Date':[group.ix[i,"Release_Date"]],'Average_Gross':[group.ix[i,"Average_Gross"]]})
                           
        director_final=director_final.append(temp)

/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [245]:
director_final = director_final.reset_index(drop=True)

In [247]:
final_data["director_gross"] = 0

for i in range(len(final_data)):
    if isinstance(final_data.ix[i,"Director"],float):
         continue
    name = final_data.ix[i,"Director"].strip("[]").split(",")
    movie = final_data.ix[i,"clean_movie"]
   
    for j in range(len(name)):
        temp = name[j].strip(" ").strip("'").strip("\"")
        
       
        if any(director_final[(director_final["name"].str.contains(temp)) & (director_final["clean_movie"] == (movie))]):
            row = director_final[(director_final["name"].str.contains(temp)) & (director_final["clean_movie"]==(movie))].index.tolist()
            if len(row) == 0:
                continue
            if len(row) > 1:
                print("More than one match",row)
            else:
                final_data.ix[i,"director_gross"] = director_final.ix[row[0],"Average_Gross"]
                    
                    
                
           



/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [249]:
director_final.to_csv("/Users/Manoj/Documents/metis/directors_final.csv")

### Reading and cleaning Producers' data

In [236]:
producers = pd.read_csv("/Users/Manoj/Documents/metis/producers.csv",converters={"movies":lambda x:x.strip("[]").split(",")})

producer_temp = pd.DataFrame(columns=["names","movies"])
for i in range(len(producers)):
    item=producers.ix[i,"movies"]
    
    
    for j in range(len(item)):
        mov = item[j].strip(" ").strip("'").strip("\"")
        temp = pd.DataFrame({'names':producers.ix[i,"names"],'movies':mov},index=[0])
        producer_temp = producer_temp.append(temp)
producer_temp = producer_temp.reset_index(drop=True)

producer_temp["clean_movie"] = producer_temp["movies"].apply(strip_punctuation)



for i in range(len(final_data)):
    item = final_data.ix[i,"clean_movie"]
    item = item.strip(" ")
    
    if any(producer_temp[producer_temp["clean_movie"].str.strip(" ") == (item)]):
        row = producer_temp[producer_temp["clean_movie"] == (item)].index.tolist()
       
        
            
        
        producer_temp.ix[row,"Gross"] = final_data.ix[i,"Gross"]
        producer_temp.ix[row,"Release_Date"] = final_data.ix[i,"Release_Date"]
        

producer_temp = producer_temp.dropna(subset=["Gross"])
producer_final = pd.DataFrame(columns=["name","Movie","Release_Date","Average_Gross","clean_movie"])
for name, group in actor_temp.groupby(actor_temp["names"]):
    
    group.sort_values(by=['Release_Date'],ascending=True,inplace=True)
    group = group.reset_index(drop=True)
    group["cum_sum"] = group["Gross"].cumsum()
    group["Average_Gross"] = [group.ix[i,'cum_sum']/(i+1) for i in range(len(group))]
    for i in range(len(group)):
        temp = pd.DataFrame({'name':[name],'clean_movie':[group.ix[i,"clean_movie"]],'Movie':[group.ix[i,"movies"]],'Gross':[group.ix[i,"Gross"]],'Release_Date':[group.ix[i,"Release_Date"]],'Average_Gross':[group.ix[i,"Average_Gross"]]})
                           
        actor_final=actor_final.append(temp)
producer_final = producer_final.reset_index(drop=True)
final_data["producer_gross"] = 0

for i in range(len(final_data)):
    if isinstance(final_data.ix[i,"Producer"],float):
         continue
    name = final_data.ix[i,"Producer"].strip("[]").split(",")
    movie = final_data.ix[i,"clean_movie"]
   
    for j in range(len(name)):
        temp = name[j].strip(" ").strip("'").strip("\"")
        
       
        if any(producer_final[(producer_final["name"].str.contains(temp)) & (producer_final["clean_movie"] == (movie))]):
            row = producer_final[(producer_final["name"].str.contains(temp)) & (producer_final["clean_movie"]==(movie))].index.tolist()
            if len(row) == 0:
                continue
            if len(row) > 1:
                print("More than one match",row)
            else:
                final_data.ix[i,"producer_gross"] = producer_final.ix[row[0],"Average_Revenue"]
                    
                    
                
           




/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:57: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:58: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [238]:
producers = pd.read_csv("/Users/Manoj/Documents/metis/producers.csv",converters={"movies":lambda x:x.strip("[]").split(",")})

producer_temp = pd.DataFrame(columns=["names","movies"])

In [239]:
for i in range(len(producers)):
    item=producers.ix[i,"movies"]
    
    
    for j in range(len(item)):
        mov = item[j].strip(" ").strip("'").strip("\"")
        temp = pd.DataFrame({'names':producers.ix[i,"names"],'movies':mov},index=[0])
        producer_temp = producer_temp.append(temp)
producer_temp = producer_temp.reset_index(drop=True)

producer_temp["clean_movie"] = producer_temp["movies"].apply(strip_punctuation)

producer_temp[:5]

,movies,names,clean_movie
0,Endless Love (2014),Pamela Abdy,EndlessLove2014
1,47 Ronin,Pamela Abdy,47Ronin
2,Identity Thief,Pamela Abdy,IdentityThief
3,Garden State,Pamela Abdy,GardenState
4,How High(executive),Pamela Abdy,HowHighexecutive


In [240]:
for i in range(len(final_data)):
    item = final_data.ix[i,"clean_movie"]
    item = item.strip(" ")
    
    if any(producer_temp[producer_temp["clean_movie"].str.strip(" ") == (item)]):
        row = producer_temp[producer_temp["clean_movie"] == (item)].index.tolist()
       
        
            
        
        producer_temp.ix[row,"Gross"] = final_data.ix[i,"Gross"]
        producer_temp.ix[row,"Release_Date"] = final_data.ix[i,"Release_Date"]

In [241]:
producer_temp[:5]

,movies,names,clean_movie,Gross,Release_Date
0,Endless Love (2014),Pamela Abdy,EndlessLove2014,23438250.0,2014-02-14
1,47 Ronin,Pamela Abdy,47Ronin,38362475.0,2013-12-25
2,Identity Thief,Pamela Abdy,IdentityThief,134506920.0,2013-02-08
3,Garden State,Pamela Abdy,GardenState,26782316.0,2004-07-28
4,How High(executive),Pamela Abdy,HowHighexecutive,NaN,NaT


In [250]:
producer_temp = producer_temp.dropna(subset=["Gross"])
producer_final = pd.DataFrame(columns=["name","Movie","Release_Date","Average_Gross","clean_movie"])
for name, group in producer_temp.groupby(producer_temp["names"]):
    
    group.sort_values(by=['Release_Date'],ascending=True,inplace=True)
    group = group.reset_index(drop=True)
    group["cum_sum"] = group["Gross"].cumsum()
    group["Average_Gross"] = [group.ix[i,'cum_sum']/(i+1) for i in range(len(group))]
    for i in range(len(group)):
        temp = pd.DataFrame({'name':[name],'clean_movie':[group.ix[i,"clean_movie"]],'Movie':[group.ix[i,"movies"]],'Gross':[group.ix[i,"Gross"]],'Release_Date':[group.ix[i,"Release_Date"]],'Average_Gross':[group.ix[i,"Average_Gross"]]})
                           
        producer_final=producer_final.append(temp)
producer_final = producer_final.reset_index(drop=True)

/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [251]:
producer_final[:5]

,Average_Gross,Gross,Movie,Release_Date,clean_movie,name
0,5.308989e+07,53089891.0,The Prestige,2006-10-20,ThePrestige,Aaron Ryder
1,2.898805e+07,4886216.0,Hamlet 2,2008-08-22,Hamlet2,Aaron Ryder
2,2.466146e+07,16008272.0,The Raven,2012-04-27,TheRaven,Aaron Ryder
3,2.639907e+07,31611916.0,House at the End of The Street,2012-09-21,HouseattheEndofTheStreet,Aaron Ryder
4,2.572372e+07,23022309.0,Transcendence,2014-04-18,Transcendence,Aaron Ryder


In [253]:
final_data["producer_gross"] = 0

for i in range(len(final_data)):
    if isinstance(final_data.ix[i,"Producer"],float):
         continue
    name = final_data.ix[i,"Producer"].strip("[]").split(",")
    movie = final_data.ix[i,"clean_movie"]
   
    for j in range(len(name)):
        temp = name[j].strip(" ").strip("'").strip("\"")
        
       
        if any(producer_final[(producer_final["name"].str.contains(temp)) & (producer_final["clean_movie"] == (movie))]):
            row = producer_final[(producer_final["name"].str.contains(temp)) & (producer_final["clean_movie"]==(movie))].index.tolist()
            if len(row) == 0:
                continue
            if len(row) > 1:
                print("More than one match",row)
            else:
                final_data.ix[i,"producer_gross"] = producer_final.ix[row[0],"Average_Gross"]
                    
                    

/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [254]:
final_data[:5]

,Release_Date,Movie,Gross,Genre,Run_Time,Rating,Budget,Num_theatres,Actors,Producer,Writer,Director,ROI,Competitors,clean_movie,Actor1,Actor2,Actor3,director_gross,producer_gross
0,2004-04-09,The Alamo (2004),22414961,Drama,137,PG-13,107000000.0,2609,"['Billy Bob Thornton', 'Dennis Quaid', 'Patric...",Ron Howard,Leslie Bohem,John Lee Hancock,-0.790514,13.0,TheAlamo2004,2.241496e+07,22414961.0,22414961.0,22414961.0,22414961.0
1,2004-12-25,Darkness,22163442,Horror,102,PG-13,10600000.0,1718,"['Anna Paquin', 'Lena Olin']",NaN,NaN,NaN,1.090891,4.0,Darkness,2.216344e+07,0.0,0.0,0.0,0.0
2,2004-01-16,Torque,21215059,Action,81,PG-13,40000000.0,2463,"['Ice Cube', 'Adam Scott']",Neal H. Moritz,NaN,NaN,-0.469624,7.0,Torque,2.121506e+07,21215059.0,0.0,0.0,21215059.0
3,2004-12-17,Flight of the Phoenix,21009180,Adventure,120,PG-13,45000000.0,2604,"['Dennis Quaid', 'Tyrese Gibson', 'Giovanni Ri...",John Davis,Scott Frank,John Moore,-0.533129,9.0,FlightofthePhoenix,7.672165e+07,21009180.0,29385928.5,21009180.0,58887000.0
4,2004-08-06,Little Black Book,20698668,Comedy,105,PG-13,35000000.0,2445,"['Brittany Murphy', 'Holly Hunter', 'Ron Livin...",NaN,Elisa Bell,Nick Hurran,-0.408609,11.0,LittleBlackBook,2.069867e+07,20698668.0,20698668.0,20698668.0,0.0


In [255]:
producer_final.to_csv("/Users/Manoj/Documents/metis/producer_final.csv")

### Reading and cleaning writers' data

In [257]:
writers = pd.read_csv("/Users/Manoj/Documents/metis/writer.csv",converters={"movies":lambda x:x.strip("[]").split(",")})

writer_temp = pd.DataFrame(columns=["names","movies"])
for i in range(len(writers)):
    item=writers.ix[i,"movies"]
    
    
    for j in range(len(item)):
        mov = item[j].strip(" ").strip("'").strip("\"")
        temp = pd.DataFrame({'names':writers.ix[i,"names"],'movies':mov},index=[0])
        writer_temp = writer_temp.append(temp)
writer_temp = writer_temp.reset_index(drop=True)

writer_temp["clean_movie"] = writer_temp["movies"].apply(strip_punctuation)



for i in range(len(final_data)):
    item = final_data.ix[i,"clean_movie"]
    item = item.strip(" ")
    
    if any(writer_temp[writer_temp["clean_movie"].str.strip(" ") == (item)]):
        row = writer_temp[writer_temp["clean_movie"] == (item)].index.tolist()
       
        
            
        
        writer_temp.ix[row,"Gross"] = final_data.ix[i,"Gross"]
        writer_temp.ix[row,"Release_Date"] = final_data.ix[i,"Release_Date"]
        

writer_temp = writer_temp.dropna(subset=["Gross"])
writer_final = pd.DataFrame(columns=["name","Movie","Release_Date","Average_Gross","clean_movie"])
for name, group in writer_temp.groupby(writer_temp["names"]):
    
    group.sort_values(by=['Release_Date'],ascending=True,inplace=True)
    group = group.reset_index(drop=True)
    group["cum_sum"] = group["Gross"].cumsum()
    group["Average_Gross"] = [group.ix[i,'cum_sum']/(i+1) for i in range(len(group))]
    for i in range(len(group)):
        temp = pd.DataFrame({'name':[name],'clean_movie':[group.ix[i,"clean_movie"]],'Movie':[group.ix[i,"movies"]],'Gross':[group.ix[i,"Gross"]],'Release_Date':[group.ix[i,"Release_Date"]],'Average_Gross':[group.ix[i,"Average_Gross"]]})
                           
        writer_final=writer_final.append(temp)
writer_final = writer_final.reset_index(drop=True)
final_data["writer_gross"] = 0

for i in range(len(final_data)):
    if isinstance(final_data.ix[i,"Writer"],float):
         continue
    name = final_data.ix[i,"Writer"].strip("[]").split(",")
    movie = final_data.ix[i,"clean_movie"]
   
    for j in range(len(name)):
        temp = name[j].strip(" ").strip("'").strip("\"")
        
       
        if any(writer_final[(writer_final["name"].str.contains(temp)) & (writer_final["clean_movie"] == (movie))]):
            row = writer_final[(writer_final["name"].str.contains(temp)) & (writer_final["clean_movie"]==(movie))].index.tolist()
            if len(row) == 0:
                continue
            if len(row) > 1:
                print("More than one match",row)
            else:
                final_data.ix[i,"writer_gross"] = writer_final.ix[row[0],"Average_Gross"]
                    
                    
                
           





/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:57: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/Users/Manoj/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:58: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [290]:
writer_final.to_csv("/Users/Manoj/Documents/metis/writer_final.csv")

In [293]:
final_data.to_csv("/Users/Manoj/Documents/metis/final_data.csv")